In [21]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 100)

data = pd.read_csv('../data/processed_data.csv')

print(f'Data shape - {data.shape}')
data.head()

Data shape - (81877, 7)


,Region,City,Location,Floor,Area,Rooms,Price
0,ivano-frankovsk,galich,"Галич, Івано-Франківська область",Middle,36.8,1,339.673913
1,ivano-frankovsk,burshtyn,"Бурштин, Івано-Франківська область",Last,50.0,3,296.000000
2,ivano-frankovsk,burshtyn,"Бурштин, Івано-Франківська область",Middle,39.4,2,380.710660
3,ivano-frankovsk,burshtyn,"Бурштин, Івано-Франківська область",Middle,50.0,1,280.000000
4,ivano-frankovsk,kalush,"Калуш, Івано-Франківська область",Middle,34.6,1,423.699422


##  My own train_test_split

In [22]:
def train_val_split(data, random_state):
    df = data.copy()
#     if df.Location:
    df.drop('Location', axis=1, inplace=True)
    
    cities = df.City.unique()
    small_np = []
    big_np = []

    for i in cities:
        if df[df.City == i].shape[0] == 1:
            small_np.append(i)

        elif df[df.City == i].shape[0] > 1:
            big_np.append(i)


    X_train1, y_train1 = df[df.City.isin(small_np)].drop('Price', axis=1), df[df.City.isin(small_np)].Price
    df = df[df.City.isin(big_np)]

    X_train, X_test, y_train, y_test = train_test_split(df.drop(['Price'], axis=1),
                                                        df.Price, test_size=0.21, random_state=random_state, stratify=df.City)

    X_train_final = pd.concat([X_train1, X_train])
    y_train_final = pd.concat([y_train1, y_train])
    
    
    return X_train_final, X_test, y_train_final, y_test

# Optuna | XGBoost pipeline

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from optuna import create_study
from xgboost import XGBRegressor

from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
def objective(trial):
    
    X_train, X_test, y_train, y_test = train_val_split(data, random_state=42)

    params = {
        'tree_method':'gpu_hist',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.2, 1.0, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.2, 1.0, 0.1),
        'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.2, 1.0, 0.1),
        'learning_rate': trial.suggest_float("learning_rate", 0.002, 0.1, log=True),
        'n_estimators': trial.suggest_int("n_estimators", 2500, 6000, 500),
        'max_depth': trial.suggest_int("max_depth", 4, 12, 1),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    model = XGBRegressor(**params)  
    
#     model.fit(X_train, y_train) #, early_stopping_rounds=100, eval_set=[(X_test, y_test)], eval_metric='rmse', verbose=False)
    
    encoder = ColumnTransformer([('encoder', OneHotEncoder(sparse_output=False), ['Region', 'Floor', 'City'])],
                        remainder='passthrough')

    pipe_optuna = Pipeline(
        steps=[
            ('encoder', encoder),
            ('model', model)
        ])
    
    pipe_optuna.fit(X_train, y_train)
    
    rmse = mean_squared_error(y_test, pipe_optuna.predict(X_test), squared=False)
    
    return rmse

In [20]:
opt_time = 60 * 60

study_xgbr = create_study(study_name="XGBRegressor Optuna Optimization", direction="minimize")
study_xgbr.optimize(objective, n_jobs=-1, timeout=opt_time)

print(f'Best value: {study_xgbr.best_value}')
# print(f'Best trial: {study.best_trial}')
print(f'Number of finished trials: {len(study_xgbr.trials)}')
print(f"Наилучшие значения гиперпараметров {study_xgbr.best_params}")

[I 2023-06-26 21:46:40,679] A new study created in memory with name: XGBRegressor Optuna Optimization
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:9: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optun

C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:9: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.sug

[I 2023-06-26 21:54:36,675] Trial 2 finished with value: 355.5829483635719 and parameters: {'lambda': 0.22808785819895017, 'alpha': 0.6474540006189071, 'subsample': 1.0, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.8, 'learning_rate': 0.002409602017058074, 'n_estimators': 4000, 'max_depth': 7, 'min_child_weight': 80}. Best is trial 0 with value: 352.91170488374087.
[I 2023-06-26 21:54:38,937] Trial 4 finished with value: 352.23647112372726 and parameters: {'lambda': 5.535521556336007, 'alpha': 0.019367934315758444, 'subsample': 0.2, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.9000000000000001, 'learning_rate': 0.08188954700961704, 'n_estimators': 2500, 'max_depth': 12, 'min_child_weight': 113}. Best is trial 4 with value: 352.23647112372726.
[I 2023-06-26 21:54:44,773] Trial 7 finished with value: 356.7712801829759 and parameters: {'lambda': 0.0016122716349590862, 'alpha': 0.023257937664538788, 'subsample': 0.4, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.9000000000000001, '

[I 2023-06-26 21:55:36,757] Trial 1 finished with value: 353.3528823273409 and parameters: {'lambda': 0.03156116028658455, 'alpha': 1.299960679526242, 'subsample': 0.5, 'colsample_bytree': 0.6000000000000001, 'colsample_bylevel': 1.0, 'learning_rate': 0.002988669905577964, 'n_estimators': 6000, 'max_depth': 6, 'min_child_weight': 22}. Best is trial 4 with value: 352.23647112372726.
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_flo

[I 2023-06-26 21:59:59,771] Trial 9 finished with value: 354.02711692485906 and parameters: {'lambda': 0.011709121858123443, 'alpha': 0.16268758227189423, 'subsample': 1.0, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.8, 'learning_rate': 0.04872088947738585, 'n_estimators': 4500, 'max_depth': 6, 'min_child_weight': 278}. Best is trial 6 with value: 351.1346741246525.
[I 2023-06-26 22:00:06,927] Trial 14 finished with value: 353.97051521548474 and parameters: {'lambda': 7.073173816098202, 'alpha': 0.030997867322956982, 'subsample': 0.9000000000000001, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.5, 'learning_rate': 0.03566454073696287, 'n_estimators': 3000, 'max_depth': 6, 'min_child_weight': 235}. Best is trial 6 with value: 351.1346741246525.
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`

...

[I 2023-06-26 22:34:59,015] Trial 40 finished with value: 353.28934743463725 and parameters: {'lambda': 0.021531189813567633, 'alpha': 0.20402372525302775, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.4, 'colsample_bylevel': 0.30000000000000004, 'learning_rate': 0.027968588968718563, 'n_estimators': 5500, 'max_depth': 8, 'min_child_weight': 31}. Best is trial 21 with value: 348.41524766577925.
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.

C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:9: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.sug

C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:9: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_discrete_uniform('subsample', 0.2, 1.0, 0.1),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:10: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  

[I 2023-06-26 22:43:36,996] Trial 50 finished with value: 351.2752128652833 and parameters: {'lambda': 0.0048982314464525245, 'alpha': 0.5440508918792534, 'subsample': 0.5, 'colsample_bytree': 0.30000000000000004, 'colsample_bylevel': 0.7, 'learning_rate': 0.018702146382134216, 'n_estimators': 4500, 'max_depth': 10, 'min_child_weight': 30}. Best is trial 21 with value: 348.41524766577925.
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\Admin\AppData\Local\Temp\ipykernel_14780\195096324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.sugg

Best value: 348.41524766577925
Number of finished trials: 60
Наилучшие значения гиперпараметров {'lambda': 0.01973328702317252, 'alpha': 6.20425684193659, 'subsample': 0.5, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.4, 'learning_rate': 0.022436637559530896, 'n_estimators': 5000, 'max_depth': 12, 'min_child_weight': 3}


# Pipeline

In [29]:
# Region City Floor Area Rooms(int64) | optuna | 60 minutes | 70 trials | xgb_v1
params = {'lambda': 0.03223912653294917, 'alpha': 0.004698857584966701, 'subsample': 1.0, 'colsample_bytree': 0.5,
          'colsample_bylevel': 0.9000000000000001, 'learning_rate': 0.05757942646087289, 'n_estimators': 2500,
          'max_depth': 12, 'min_child_weight': 1}

# Region City Floor Area Rooms(int64) Offers | optuna | 60 minutes | 60 trials | xgb_v3
# params = {'lambda': 0.01973328702317252, 'alpha': 6.20425684193659, 'subsample': 0.5, 'colsample_bytree': 0.5,
#           'colsample_bylevel': 0.4, 'learning_rate': 0.022436637559530896, 'n_estimators': 5000, 'max_depth': 12,
#           'min_child_weight': 3}



xgb_model = XGBRegressor(**params) 


enc = ColumnTransformer([('encoder', OneHotEncoder(sparse_output=False), ['Region', 'Floor', 'City'])],
                        remainder='passthrough')

pipe = Pipeline(
    steps=[
        ('encoder', enc),
        ('model', xgb_model)
    ])

# Training pipeline

In [31]:
%%time

X_train, X_test, y_train, y_test = train_val_split(data, random_state=42)

pipe.fit(X_train, y_train)
print(f'rmse = {mean_squared_error(pipe.predict(X_test)*X_test.Area, y_test*X_test.Area, squared=False)}')

rmse = 21313.43496451753
CPU times: total: 2h 45min 30s
Wall time: 22min 30s


# Train on the full dataset

In [33]:
pipe.fit(data.drop(['Price', 'Location'], axis=1), data['Price'])

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Region', 'Floor',
                                                   'City'])])),
                ('model',
                 XGBRegressor(alpha=0.004698857584966701, base_score=None,
                              booster=None, callbacks=None,
                              colsample_bylevel=0.9000000000000001,
                              colsample_bynode=None, colsample_bytree=0.5,
                              early_stopping_rou...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None,
                              lambda=0.03223912653294917,
                              learning_rate=0.05757942646087289, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=12,
                              max_leaves=None, min_child_weight=1, missing=nan,
                              monotone_constraints=None, n_estimators=2500,
                              n_jobs=None, num_parallel_tree=None, ...))])

# Save model

In [35]:
import joblib
import pickle

joblib.dump(pipe, '../models/xgb_v1.joblib')
# joblib.dump(pipe1, '../models/xgb_v1.pkl')
# s = pickle.dumps(pipe1)

['../models/xgb_v1.joblib']